<a href="https://colab.research.google.com/github/HSS107048212/Burnout-Analysis-and-Prediction/blob/main/112034602_%E9%99%B3%E7%9A%86%E9%8C%AB_HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

參考資料:

https://www.kaggle.com/code/willribeiro/burnout-analysis-and-prediction

https://www.kaggle.com/code/ushasridevarakonda/employee-burnout-prediction

https://www.kaggle.com/code/blurredmachine/employee-burn-rate-analysis-eda-viz-pred







### Import Package

In [1]:
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split
# Category Encoder (categorical variable)
from sklearn.preprocessing import LabelEncoder
# Min max normalization (numerical variable)
from sklearn.preprocessing import MinMaxScaler
# Model (can also view the scikit learn to choose the model)
from sklearn.linear_model import LinearRegression
os.getcwd()

'/content'

In [2]:
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import scipy.stats as st

### Read data

In [30]:
df = pd.read_csv(r'./train.csv')
print('viwe the data shape', df.shape)
df.head()

viwe the data shape (10108, 9)


,Employee ID,Date of Joining,Gender,Company Type,WFH Setup Available,Designation,Resource Allocation,Mental Fatigue Score,Burn Rate
0,0,2008/9/30,Female,Service,No,2,3.0,3.8,0.16
1,1,2008/11/30,Male,Service,Yes,1,2.0,5.0,0.36
2,2,2008/3/10,Female,Product,Yes,2,NaN,5.8,0.49
3,3,2008/11/3,Male,Service,Yes,1,1.0,2.6,0.20
4,4,2008/7/24,Female,Service,No,3,7.0,6.9,0.52


### Data Preprocessing

In [31]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score

# 1. 数据预处理
# 用均值替代缺失值
imputer = SimpleImputer(strategy='mean')
df['Mental Fatigue Score'].fillna(df['Mental Fatigue Score'].mean(), inplace=True)
df['Designation'].fillna(df['Designation'].mean(), inplace=True)
df['Resource Allocation'].fillna(df['Resource Allocation'].mean(), inplace=True)
df['Burn Rate'].fillna(df['Burn Rate'].mean(), inplace=True)
df.drop(["Employee ID", "Date of Joining"], axis=1, inplace=True)


# 类别特征编码，例如使用独热编码
df = pd.get_dummies(df, columns=['Gender', 'Company Type', 'WFH Setup Available'])

# 2. 数据分割
X = df.drop("Burn Rate", axis=1)  # 特征
y = df["Burn Rate"]  # 目标
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.9, random_state=42)

# 3. 建立模型
# 决策树回归
decision_tree_model = DecisionTreeRegressor()
# 随机森林回归
random_forest_model = RandomForestRegressor()
# 支持向量回归
svr_model = SVR()

# 4. 模型训练
decision_tree_model.fit(X_train, y_train)
random_forest_model.fit(X_train, y_train)
svr_model.fit(X_train, y_train)

# 5. 模型评估
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print(f"均方误差 (MSE): {mse}")
    print(f"R平方 (R^2): {r2}")

print("Decision Tree Model:")
evaluate_model(decision_tree_model, X_test, y_test)

print("\nRandom Forest Model:")
evaluate_model(random_forest_model, X_test, y_test)

print("\nSVR Model:")
evaluate_model(svr_model, X_test, y_test)


Decision Tree Model:
均方误差 (MSE): 0.010813803329706557
R平方 (R^2): 0.7131595884859578

Random Forest Model:
均方误差 (MSE): 0.006870960164223273
R平方 (R^2): 0.8177450633313972

SVR Model:
均方误差 (MSE): 0.006168969037588705
R平方 (R^2): 0.8363656556894912


### Training (model construction)

### Prediction
1. read data
2. model prediction
3. to_csv

In [ ]:
df_test = pd.read_csv(r'./test.csv')
# df_test = df_test.dropna()
a = list(df_test)
for i in range(len(a)):
    df_test[a[i]] = df_test[a[i]].fillna(0)
df_test = df_test.drop(['Employee ID','Date of Joining'],axis=1)
print(df_test.shape)
df_test.head()

### Same preprocessing step as training procedure
- can write it like a function or pipeline

In [ ]:
df_test['Mental Fatigue Score'].fillna(df_test['Mental Fatigue Score'].mean(), inplace=True)
df_test['Designation'].fillna(df_test['Designation'].mean(), inplace=True)
df_test['Resource Allocation'].fillna(df_test['Resource Allocation'].mean(), inplace=True)

# 类别特征编码，例如使用独热编码
#df_test = pd.get_dummies(df_test, columns=['Gender', 'Company Type', 'WFH Setup Available'])

X_test = df_test[df_test.columns]

In [ ]:
y_pred_decision_tree_model = decision_tree_model.predict(X_test)
y_pred_random_forest_model = random_forest_model.predict(X_test)
y_pred_svr_model = svr_model.predict(X_test)

In [ ]:
# 将小于1的数值转换为0
y_pred_decision_tree_model[y_pred_decision_tree_model < 0] = 0
# 将小于1的数值转换为0
y_pred_random_forest_model[y_pred_random_forest_model < 0] = 0
# 将小于1的数值转换为0
y_pred_svr_model[y_pred_svr_model < 0] = 0

### Output .csv for submission

In [ ]:
df_ans = pd.DataFrame(y_pred_decision_tree_model.astype(float), columns=['Burn Rate'])
df_ans.to_csv('mySubmission.csv', index_label='Employee ID')
df_ans = pd.DataFrame(y_pred_random_forest_model.astype(float), columns=['Burn Rate'])
df_ans.to_csv('mySubmission1.csv', index_label='Employee ID')
df_ans = pd.DataFrame(y_pred_svr_model.astype(float), columns=['Burn Rate'])
df_ans.to_csv('mySubmission2.csv', index_label='Employee ID')

### Some Tips
1. try different scaling approach (numerical variable)
2. encoding method
3. different model
4. feature engineering (some different transformation)
5. do some data exploration to see other pattern
6. hyperparameter

### Other Complementary
- pipeline: https://scikit-learn.org/stable/modules/compose.html#pipeline
- also can try some basic deep learning model (ANN)